# Question 1: 
Given some sample data, write a program to answer the following: click here to access the required data set

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 
What metric would you report for this dataset?
What is its value?


In [9]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

sneaker_df = pd.read_csv("dataset.csv")

sneaker_df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [10]:
sneaker_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        5000 non-null   int64 
 1   shop_id         5000 non-null   int64 
 2   user_id         5000 non-null   int64 
 3   order_amount    5000 non-null   int64 
 4   total_items     5000 non-null   int64 
 5   payment_method  5000 non-null   object
 6   created_at      5000 non-null   object
dtypes: int64(5), object(2)
memory usage: 273.6+ KB


In [11]:
sneaker_df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


In [12]:
sneaker_df.columns

Index(['order_id', 'shop_id', 'user_id', 'order_amount', 'total_items',
       'payment_method', 'created_at'],
      dtype='object')

In [15]:
sneaker_df_sorted = sneaker_df.sort_values(by='created_at').reset_index(drop = True)
sneaker_df_sorted.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1863,39,738,536,4,cash,2017-03-01 0:08:09
1,1742,39,910,268,2,cash,2017-03-01 0:10:19
2,3229,97,912,324,2,cash,2017-03-01 0:14:12
3,1268,80,798,290,2,credit_card,2017-03-01 0:19:31
4,2690,49,799,258,2,credit_card,2017-03-01 0:22:25


In [21]:
print(f"Unique shops: {sneaker_df['shop_id'].nunique()} and Unique users: {sneaker_df['user_id'].nunique()}")

Unique shops: 100 and Unique users: 301


In [22]:
np.mean(sneaker_df['order_amount']/sneaker_df['total_items'])

387.7428

In [23]:
np.mean(sneaker_df['order_amount'])

3145.128